In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Sequential
from keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Flatten, TimeDistributed, ConvLSTM2D, Reshape
import tensorflow as tf
import sklearn.metrics as sm
import keras
from keras.losses import huber_loss


from keras import backend as K

Using TensorFlow backend.


In [2]:
data = pd.read_csv("train.csv")

In [3]:
dtest = pd.read_csv("test.csv")

In [4]:
data.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [5]:
data =  data.drop(["ID","X0","X1","X2","X3","X4","X5","X6","X8" ], axis = 1)

In [6]:
data.head()

,y,X10,X11,X12,X13,X14,X15,X16,X17,X18,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,130.81,0,0,0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
1,88.53,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
2,76.26,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,80.62,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,78.02,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
data.shape

(4209, 369)

In [8]:
dtest

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,8410,aj,h,as,f,d,aa,j,e,0,...,0,0,0,0,0,0,0,0,0,0
4205,8411,t,aa,ai,d,d,aa,j,y,0,...,0,1,0,0,0,0,0,0,0,0
4206,8413,y,v,as,f,d,aa,d,w,0,...,0,0,0,0,0,0,0,0,0,0
4207,8414,ak,v,as,a,d,aa,c,q,0,...,0,0,1,0,0,0,0,0,0,0


In [9]:
dtest =  dtest.drop(["ID","X0","X1","X2","X3","X4","X5","X6","X8" ], axis = 1)

In [10]:
dtest.head() 

,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [11]:
dtest.shape

(4209, 368)

In [12]:
X_train, y_train = data.values[0:3499,1:], data.values[0:3499, :1].ravel()
X_valid, y_valid = data.values[3500:4208,1:], data.values[3500:4208, :1].ravel()
X_test = dtest.values[:, :]

In [13]:
print("X_train----",X_train.shape)
print("y_train----",y_train.shape)      
print("X_valid----",X_valid.shape)
print("y_valid----",y_valid.shape)
print("X_test-----",X_test.shape)

X_train---- (3499, 368)
y_train---- (3499,)
X_valid---- (708, 368)
y_valid---- (708,)
X_test----- (4209, 368)


In [14]:
X_train[0]

array([0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
       0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 0., 0.

In [15]:
y_train[0]

130.81

In [16]:
# y_train = min_max_scaler.fit_transform(y_train)


In [17]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [18]:
def metrics(pred, y_test):
    evs = sm.explained_variance_score(y_test, pred)
    me = sm.max_error(y_test, pred)
    mae = sm.mean_absolute_error(y_test, pred)
    mse = sm.mean_squared_error(y_test, pred)
    rmse = np.sqrt(mse)
    #msle = sm.mean_squared_log_error(y_test, pred)
    m_ae = sm.median_absolute_error(y_test, pred)
    r2 = sm.r2_score(y_test, pred)
    #mpd = sm.mean_poisson_deviance(y_test, pred)
    #mgd = sm.mean_gamma_deviance(y_test, pred)
    mape = mean_absolute_percentage_error(pred, y_test)
    return({'Explained Variance Score': evs,
            'Max Error': me,
            'Mean Absolute Error': mae,
            'Mean Squared Error': mse,
            'Root Mean Squared Error': rmse,
            #'Mean Squared Log Error': msle,
            'Median Absolute Error': m_ae,
            'R² Score': r2,
            #'Mean Poisson Deviance': mpd,
            #'Mean Gamma Deviance': mgd,
            'Mean Absolute Percentage Error': mape
            })

In [19]:
subsequences = 2
timesteps = X_train.shape[1]//subsequences

In [20]:
timesteps 

184

In [21]:
X_train = X_train.reshape((X_train.shape[0], subsequences, 1, timesteps, 1))
X_valid = X_valid.reshape((X_valid.shape[0], subsequences, 1, timesteps, 1))
X_test  = X_test.reshape((X_test.shape[0], subsequences, 1, timesteps, 1))

In [22]:
X_train.shape

(3499, 2, 1, 184, 1)

In [23]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=50)

In [24]:
modelConvLSTM = Sequential()
modelConvLSTM.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', return_sequences=True,input_shape=(X_train.shape[1], 1, X_train.shape[3], X_train.shape[4])))
modelConvLSTM.add(ConvLSTM2D(filters=32, kernel_size=(1,2), activation='relu'))
modelConvLSTM.add(Flatten())
modelConvLSTM.add(Dense(64))
modelConvLSTM.add(Dense(32))
modelConvLSTM.add(Dense(1))
modelConvLSTM.compile(optimizer='adam', loss='huber_loss')
history = modelConvLSTM.fit(X_train, y_train, batch_size=512, epochs=700, verbose=2, callbacks=[early_stop], validation_data = (X_valid, y_valid))

Train on 3499 samples, validate on 708 samples
Epoch 1/700
 - 67s - loss: 99.9378 - val_loss: 96.3830
Epoch 2/700
 - 47s - loss: 94.6588 - val_loss: 84.5971
Epoch 3/700
 - 47s - loss: 73.0430 - val_loss: 43.4973
Epoch 4/700
 - 47s - loss: 25.2121 - val_loss: 28.4330
Epoch 5/700
 - 47s - loss: 16.4671 - val_loss: 14.5747
Epoch 6/700
 - 48s - loss: 13.6031 - val_loss: 10.1783
Epoch 7/700
 - 46s - loss: 11.2562 - val_loss: 8.8845
Epoch 8/700
 - 46s - loss: 10.3438 - val_loss: 8.9709
Epoch 9/700
 - 46s - loss: 9.7981 - val_loss: 9.6385
Epoch 10/700
 - 47s - loss: 9.7467 - val_loss: 8.9028
Epoch 11/700
 - 47s - loss: 9.5906 - val_loss: 9.0960
Epoch 12/700
 - 46s - loss: 9.5703 - val_loss: 8.7840
Epoch 13/700
 - 46s - loss: 9.4975 - val_loss: 8.8632
Epoch 14/700
 - 46s - loss: 9.4670 - val_loss: 8.7544
Epoch 15/700
 - 46s - loss: 9.4209 - val_loss: 8.7532
Epoch 16/700
 - 47s - loss: 9.3968 - val_loss: 8.7103
Epoch 17/700
 - 46s - loss: 9.3694 - val_loss: 8.6838
Epoch 18/700
 - 47s - loss: 9.

Epoch 151/700
 - 46s - loss: 4.8733 - val_loss: 4.2141
Epoch 152/700
 - 46s - loss: 4.8760 - val_loss: 4.7314
Epoch 153/700
 - 47s - loss: 4.8898 - val_loss: 4.1044
Epoch 154/700
 - 46s - loss: 4.9051 - val_loss: 4.7034
Epoch 155/700
 - 46s - loss: 4.8765 - val_loss: 4.5337
Epoch 156/700
 - 46s - loss: 4.8917 - val_loss: 4.1091
Epoch 157/700
 - 46s - loss: 4.9240 - val_loss: 4.5149
Epoch 158/700
 - 46s - loss: 4.8902 - val_loss: 4.7397
Epoch 159/700
 - 46s - loss: 4.8792 - val_loss: 4.3269
Epoch 160/700
 - 47s - loss: 4.8779 - val_loss: 4.9842
Epoch 161/700
 - 46s - loss: 5.0267 - val_loss: 4.0460
Epoch 162/700
 - 46s - loss: 5.0991 - val_loss: 4.5473
Epoch 163/700
 - 46s - loss: 4.8467 - val_loss: 5.0089
Epoch 164/700
 - 47s - loss: 4.9183 - val_loss: 4.1052
Epoch 165/700
 - 47s - loss: 4.8657 - val_loss: 4.7201
Epoch 166/700
 - 46s - loss: 4.8371 - val_loss: 4.1878
Epoch 167/700
 - 47s - loss: 4.8718 - val_loss: 4.6812
Epoch 168/700
 - 46s - loss: 4.8886 - val_loss: 4.1965
Epoch 169/

Epoch 300/700
 - 46s - loss: 4.7440 - val_loss: 4.6928
Epoch 301/700
 - 46s - loss: 4.7666 - val_loss: 4.2849
Epoch 302/700
 - 47s - loss: 4.7186 - val_loss: 4.0888
Epoch 303/700
 - 46s - loss: 4.7044 - val_loss: 4.9240
Epoch 304/700
 - 47s - loss: 4.7396 - val_loss: 4.0893
Epoch 305/700
 - 46s - loss: 4.6814 - val_loss: 4.3994
Epoch 306/700
 - 46s - loss: 4.6865 - val_loss: 4.2288
Epoch 307/700
 - 46s - loss: 4.6745 - val_loss: 4.1865
Epoch 308/700
 - 46s - loss: 4.6699 - val_loss: 4.2432
Epoch 309/700
 - 46s - loss: 4.7208 - val_loss: 4.2784
Epoch 310/700
 - 46s - loss: 4.6662 - val_loss: 4.3595
Epoch 311/700
 - 46s - loss: 4.6668 - val_loss: 4.2686
Epoch 312/700
 - 46s - loss: 4.6743 - val_loss: 4.0257
Epoch 313/700
 - 46s - loss: 4.7414 - val_loss: 4.8866
Epoch 314/700
 - 46s - loss: 4.8916 - val_loss: 3.9526
Epoch 315/700
 - 46s - loss: 4.9169 - val_loss: 5.0764
Epoch 316/700
 - 47s - loss: 4.7977 - val_loss: 4.0950
Epoch 317/700
 - 46s - loss: 4.7321 - val_loss: 4.0154
Epoch 318/

In [25]:
ConvLSTMpred = modelConvLSTM.predict(X_valid, verbose=0)
ConvLSTMpred = ConvLSTMpred.reshape((ConvLSTMpred.shape[0]))
ConvLSTMresults = metrics(ConvLSTMpred, y_valid)
ConvLSTMresults

{'Explained Variance Score': 0.6681566977322875,
 'Max Error': 43.57510864257813,
 'Mean Absolute Error': 5.071442193500067,
 'Mean Squared Error': 45.701804261837204,
 'Root Mean Squared Error': 6.760310958960187,
 'Median Absolute Error': 4.331775360107422,
 'R² Score': 0.653021377756974,
 'Mean Absolute Percentage Error': 5.048727175214505}

In [26]:
ConvLSTMpred = modelConvLSTM.predict(X_test, verbose=0)
ConvLSTMpred = ConvLSTMpred.reshape((ConvLSTMpred.shape[0]))


In [27]:
ConvLSTMpred 

array([ 78.36369 ,  90.270706,  77.05674 , ...,  94.3286  , 110.587555,
        93.061295], dtype=float32)